### Import

In [1]:
import os
import time
import datetime
import numpy as np
import pandas as pd
import requests
import json
import csv

import pymysql

from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import quote
from urllib.request import urlretrieve
from collections import Counter

### ENV

In [2]:
# Load secrets.json
with open('../src/secrets.json') as f:

    json_data = json.load(f)

#####################################################
BUCKET_NAME = json_data['BUCKET_NAME']
REGION_NAME = json_data['BUCKET_REGION']
ACCESS_KEY = json_data['AWS_ACCESS_KEY']
SECRET_KEY = json_data['AWS_SECRET_KEY']
#####################################################
DB_USER_NAME = json_data['MYSQL_USERNAME']
DB_USER_PASSWORD= json_data['MYSQL_PASSWORD']
DB_HOST = json_data['MYSQL_HOST']
DB_TABLE_NAME = json_data['MYSQL_DB_NAME']
#####################################################

In [3]:
def get_exhibition_url(main_url):
    print(f'url : {main_url} 을 크롤링 합니다.')
        
    resp = requests.get(main_url)

    driver.get (main_url)

    # 접근한 페이지의 소스코드를 텍스트로 저장 (Str)
    page_text = driver.page_source

    # page_text를 html list로 변환 
    html = BeautifulSoup(page_text, 'html.parser')

    global url_list
    
    url_list=[]

    total_page_num = int(html.select_one('div.cm_paging_area._page > div > span > span._total').text)
    
    print(f"총 {total_page_num} 페이지가 있습니다.")

    for i in tqdm(range(total_page_num)):
        page_text = driver.page_source
        html = BeautifulSoup(page_text, 'html.parser')

        for i in html.select('div.area_text_box > strong.this_text > a'):
            urls = i.attrs['href']
            url_list.append('https://search.naver.com/search.naver' + urls)

        driver.find_element_by_xpath('//*[@id="main_pack"]/div[2]/div[2]/div/div/div[3]/div/a[2]').click()
        time.sleep(1)

    print(f"전시회{len(url_list)}개의 주소를 수집했습니다.")
    
    # print(url_list)

In [4]:
def get_exhibition_info(element):
    
  # print(f'url : {url}')
    
  resp = requests.get(element)
    
  # print(f'resp : {resp}')
    
  driver.get(element)

  page_text = driver.page_source

  html = BeautifulSoup(page_text, 'html.parser')

  # get title
  title = html.select_one('div.title_area._title_area > h2 > span.area_text_title > strong > a').text
  
  # get place
  try:
    place = html.select_one('div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div > div > div.detail_info > dl > div:nth-child(4) > dd > a').text
    
  except:
    place = html.select_one('div.sc_new.cs_common_module.case_normal.color_23._kgs_art_exhibition > div.cm_content_wrap > div > div > div.detail_info > dl > div:nth-child(3) > dd > a').text

  # get starting date, ending date
  try:
    date = html.select_one('div.detail_info > dl.info > div:nth-child(2) > dd').text
    startAt = date.split(' ~')[0]
    endAt = date.split(' ~')[1]
  except:
    startAt = '정보 없음'  
    endAt = '정보 없음'

  # get detail information url
  try: 
    detail_info_url = html.select_one('span.area_text_title > strong > a').attrs['href']
  except: 
    detail_info_url = '정보 없음'
  
  # ----------------------------------------------------------------------------------
  # append
  title_list.append(title)
  place_list.append(place)
  startAt_list.append(startAt)
  endAt_list.append(endAt)
  detail_info_url_list.append(detail_info_url)

In [7]:
def get_DataFrame():

  data = {
    'name': title_list,
    'start_at': startAt_list,
    'end_at': endAt_list,
    'price' : '무료',
    'gallery_location': place_list,
    'detail_info_url': detail_info_url_list 
  }

  global df

  df = pd.DataFrame(data)

In [5]:
def save_csv() :
    # 현재 시간 기준 폴더 생성
    # NOW = datetime.datetime.now().strftime('%Y-%m-%d/%H:%M:%S')
    global NOW
    NOW = datetime.datetime.now().strftime('%Y-%m-%d') + '/csv'
    # RECORD_IMG_PATH = os.path.join(f'../../data/img', NOW)
    RECORD_CSV_PATH = os.path.join('../../data', NOW)
    os.makedirs(RECORD_CSV_PATH, exist_ok=True)
    
    # df.to_csv(f'{RECORD_CSV_PATH}/exhibition.csv', index=False, encoding='utf-8-sig')
    
    # header 없이 저장
    df.to_csv(f'{RECORD_CSV_PATH}/free_exhibition.csv', index=False, encoding='utf-8-sig', header=False)

### RUN

In [8]:
# 크롬 드라이버 생성
driver = webdriver.Chrome(executable_path = '../chromedriver')

# 크롤링 웹사이트 url 설정
main_url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회'

try:
    get_exhibition_url(main_url)

    # title
    title_list = []

    # place
    place_list = []

    # date
    startAt_list = []
    endAt_list = []

    # detail_info_url_list
    detail_info_url_list = []

    print(f'{len(url_list)} 개의 전시회 세부 정보를 수집합니다.')
    for element in tqdm(url_list) :
        get_exhibition_info(element)
    time.sleep(1)
    
    driver.close()
    
    get_DataFrame()
    
    save_csv()
    
except Exception as e:
    print(e)
    print('Error Accured')
    driver.close()

url : https://search.naver.com/search.naver?where=nexearch&sm=tab_etc&mra=bjBC&qvt=0&query=무료%20전시회 을 크롤링 합니다.
총 14 페이지가 있습니다.


100%|██████████| 14/14 [00:23<00:00,  1.70s/it]


전시회53개의 주소를 수집했습니다.
53 개의 전시회 세부 정보를 수집합니다.


100%|██████████| 53/53 [00:58<00:00,  1.11s/it]


### Insert Data to DB

In [9]:
# DB 연결
connection = pymysql.connect(
    user = DB_USER_NAME,
    password = DB_USER_PASSWORD,
    host = DB_HOST,
    db = DB_TABLE_NAME,
    charset = 'utf8',
)

In [10]:
# 커서 획득
cursor = connection.cursor(pymysql.cursors.DictCursor)

In [12]:
NOW_DATE = datetime.datetime.now().strftime('%Y-%m-%d')

NOW_DATE

'2023-01-28'

In [14]:
# CSV

NOW_DATE = datetime.datetime.now().strftime('%Y-%m-%d')

f = open(f'../../data/{NOW_DATE}/csv/free_exhibition.csv', 'r', encoding='utf-8-sig')

csvReader = csv.reader(f)

In [15]:
# 컬럼 매핑

for row in csvReader:
    name = (row[0])
    start_at = (row[1])
    end_at = (row[2])
    price = (row[3])
    gallery_location = (row[4])
    detail_info = (row[5])
    
    insert_sql = "INSERT INTO exhibition (name, start_at, end_at, price, detail_info, gallery_location) VALUES (%s, %s, %s, %s, %s, %s)"
    
    cursor.execute(insert_sql, (name, start_at, end_at, price, detail_info, gallery_location))
    


DataError: (1406, "Data too long for column 'start_at' at row 1")

In [17]:
df.head()

,name,start_at,end_at,price,gallery_location,detail_info_url
0,Visualizing the invisible : 일상화된 건축의 관찰과 기록,2023.01.27.(금),2023.03.31.(금),무료,동대문디자인플라자갤러리문,https://ddp.or.kr/index.html?menuno=230&siteno...
1,이현정 개인전,2023.01.26.(목),2023.02.26.(일),무료,코소,https://blog.naver.com/coso_seoul/222990149581
2,갤러리탐 51기 : 감영주 개인전,2023.01.04.(수),2023.04.03.(월),무료,탐앤탐스 창동본점,https://shopping.naver.com/art/stores/10025441...
3,갤러리탐 51기 : 토이아치 개인전,2023.01.04.(수),2023.04.03.(월),무료,탐앤탐스 블랙그레이트점,https://shopping.naver.com/art/stores/10025441...
4,갤러리탐 51기 : 이이영 개인전,2023.01.04.(수),2023.04.03.(월),무료,탐앤탐스 탐스커버리 건대점,https://shopping.naver.com/art/stores/10025441...
